In [8]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

--2022-03-08 00:08:23--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg.1’

La_noche_estrellada 100%[===================>] 218.48K   330KB/s    in 0.7s    

2022-03-08 00:08:25 (330 KB/s) - ‘La_noche_estrellada1.jpg.1’ saved [223725/223725]



In [9]:
# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

--2022-03-08 00:08:25--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [image/jpeg]
Saving to: ‘775px-Neckarfront_Tübingen_Mai_2017.jpg’

775px-Neckarfront_T 100%[===================>] 149.43K   284KB/s    in 0.5s    

2022-03-08 00:08:27 (284 KB/s) - ‘775px-Neckarfront_Tübingen_Mai_2017.jpg’ saved [153015/153015]



In [10]:
# Creamos el directorio para los archivos de salida
!mkdir ./content/output

mkdir: cannot create directory ‘./content/output’: No such file or directory


In [11]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [12]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("./775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("./La_noche_estrellada1.jpg")
result_prefix = Path("./content/output")
iterations = 100

In [13]:
# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

In [14]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

# Respuesta:
# La funcion loss que esta minimizando el paper se compone de dos elementos:
# el elemento que representa el contenido de la imagen y el elemento que representa el estilo de la imagen
# Entonces dependiendo de la proporcion de cada elemento la imagen resultante sera mejor o peor
# si se le da mas peso al contenido, la imagen tendra pocos rasgos del estilo de la segunda imagen
# en cambio si se da mas peso al estilo, se podria perder la estructura inicial de la primera imagen siendo irreconocible.
# total_variation_weight tiene la relacion entre content_weight/style_weight
# alfa (content_weight)/ beta (style_weight)

In [15]:
# hack to error: Could not import PIL.Image. The use of `load_img` requires PIL.
# uninstall pillow from conda and install by pip
# NO LO USE
#import sys
#from PIL import Image
#sys.modules['Image'] = Image 

#from PIL import Image
#print(Image.__file__)

#import Image
#print(Image.__file__)

In [16]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [17]:
# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?
#Ayuda: https://keras.io/applications/ => https://keras.io/api/applications/vgg/#vgg19-function

#Respuesta:
# cada keras application espera un tipo especifico de formato y tamano del vector
# por ejemplo vgg19 requiere que las imagenes esten en BGR
# y no en RGB. y centrara en zero cada canal de color con respecto al dataset ImageNet.
# img # (400, 517, 3)
# img after preprocess # (1, 400, 517, 3)

In [18]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [19]:

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

# Respuesta:
# como las imagenes para ser usadas en VGG19 necesitan un formato especifico BGR y la normalizacion centrada en 0.
# la salida necesita tener el  proceso inverso para obtener la imagen en el formato RGB, y los valores correctos en los canales.

In [20]:

def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [21]:
# style image
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

2022-03-08 00:08:53.060193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 00:08:53.061461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-08 00:08:53.061572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-08 00:08:53.061669: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-08 00:08:53.061772: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [22]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

In [23]:
#Aclaración:

#La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [24]:

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

  <tf.Variable 'Variable:0' shape=(1, 400, 517, 3) dtype=float32>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.


In [25]:

# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


In [66]:
layer_features2 = outputs_dict['block1_conv1'] # 400,517,3 or 64
# layer_features2[1, :, :, :] 
# layer_features2[2, :, :, :] 
layer_features2

<KerasTensor: shape=(3, 400, 517, 64) dtype=float32 (created by layer 'block1_conv1')>

In [65]:
outputs_dict['block1_conv1'][2] # 400,517,3 or 64

<KerasTensor: shape=(400, 517, 64) dtype=float32 (created by layer 'tf.__operators__.getitem_29')>

In [50]:
outputs_dict['block2_conv1'][0]

<KerasTensor: shape=(200, 258, 128) dtype=float32 (created by layer 'tf.__operators__.getitem_14')>

In [52]:
outputs_dict['block3_conv1'][2]

<KerasTensor: shape=(100, 129, 256) dtype=float32 (created by layer 'tf.__operators__.getitem_16')>

In [54]:
outputs_dict['block4_conv1'][2]

<KerasTensor: shape=(50, 64, 512) dtype=float32 (created by layer 'tf.__operators__.getitem_18')>

In [46]:
outputs_dict['block5_conv1'][2]

<KerasTensor: shape=(25, 32, 512) dtype=float32 (created by layer 'tf.__operators__.getitem_10')>

In [26]:
# 4) En la siguientes celdas:

#- ¿Qué es la matriz de Gram?¿Para qué se usa?

#Es una matriz que contiene el producto punto entre dos vectores para extraer el estilo (textura) de una imagen, esto indica cuan similar son estos dos vectores, para luego
#calcular la funcion de optimizacion "style loss". Este producto punto devuelve informacion sobre la textura de la imagen y
#Zero informacion sobre su estructura espacial (Contenido).

#Gram matrix es el resultado del producto punto de un vector que representa "features maps" de una cnn.
#un feature map puede ser textures, brush strokes, lines, curves, dots, color distribution.
#Esto usa vectores aplanados de tamano C a partir de la capa de una cnn de profundidad C. (CxC)
# Para cada layer se genera una gram matrix entonces de esto se calcula el mse loss, y sumariza por cada capa.

# En sintesis Gram matrix se usa para capturar el estilo a partir de caracteristicas "features map / features metrics" de una CNN.

#- ¿Por qué se permutan las dimensiones de x?
# Para tener una estructura donde se pueda realizar el producto punto.
# x consiste en un tensor donde la posicion 2 corresponde a un batch de 64
# la posicion 0 e 1 es la altura y el ancho de la imagen.

In [27]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [ ]:

# 5) Losses:

#Explicar qué mide cada una de las losses en las siguientes tres celdas.

#Rta:

In [ ]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [28]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [29]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [ ]:

# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [ ]:

grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

In [ ]:

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

#Respuesta:

In [ ]:

def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.


In [ ]:

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [ ]:
# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:

evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

In [ ]:

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:
